# DnCNN_V1

Implementing the "Beyond Gaussian Denoising" Neural Network.

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gc

## Data Preprocessing

I will start by loading two of the images in. Then I will select from the originals each only one. Aftwards, I will select the 500 images in good and bad quality from the two image and create the classification label for it.

In [80]:
select = np.random.choice(np.arange(0, 4055), size = 100)

In [81]:
X_1 = np.load('/home/cloud/MRT_Data/unziped/out/P2_X.npy')

X_2 = np.load('/home/cloud/MRT_Data/unziped/out/P11_X.npy')

X_3 = np.load('/home/cloud/MRT_Data/unziped/out/P6_X.npy')

X = np.concatenate(seq = (X_1, X_2), axis=0)

X.shape

del X_1, X_2

In [82]:
X = X_1

In [83]:
X = X.reshape([4055, 256, 256, 1])

In [84]:
X = X[select, :, :]

In [85]:
del X_1

In [86]:
gc.collect()

27969

 Normalize data

In [87]:
Y_1 = np.load('/home/cloud/MRT_Data/unziped/out/P2_Y.npy')

Y_2 = np.load('/home/cloud/MRT_Data/unziped/out/P11_Y.npy')

Y_3 = np.load('/home/cloud/MRT_Data/unziped/out/P6_Y.npy')

Y = np.concatenate(seq = (Y_1, Y_2), axis=0)

Y.shape

del Y_1, Y_2

In [88]:
Y = Y_1

In [89]:
Y = Y.reshape([4055, 256, 256, 1])

In [90]:
Y.shape

(4055, 256, 256, 1)

In [91]:
Y = Y[select, :, :]

clean up the mess

In [92]:
gc.collect()

3

# Create train and test and validation set

In [93]:
train_frac = 0.9

In [94]:
train_index = int(train_frac * Y.shape[0])

In [95]:
X_train = X[0:train_index,:,:,:]
X_eval = X[train_index:X.shape[0],:,:,:]

In [96]:
Y_train = Y[0:train_index,:,:]
Y_eval = Y[train_index:X.shape[0],:,:,:]

In [97]:
def conv_layer(tensor_in, name_layer, is_training):
    x = tf.layers.conv2d(
        inputs = tensor_in,
        filters = 64,
        kernel_size = [3, 3],
        padding = "same",
        activation= None,
        name = name_layer)
    
    x = tf.layers.batch_normalization(x, name = name_layer + "_bn",
                                             center=True, 
                                             scale=True, 
                                             training=is_training)
    
    return tf.nn.relu(x, name = name_layer + "_relu")

# Create and train CNN

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    ## Hyper paramters ##
    eps_start = 0.1 #learning rate in the beginning
    eps_end = eps_start / 100 #final learning rate
    tau = 10000 # number of iterations afterwards is the learning rate constant
    #####################
    
    # Input Layer
    input_layer = features['x']
    
    # Convolutional layer #1     
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 64,
        kernel_size = 3,
        padding = "same",
        activation= tf.nn.relu,
        name = "Conv_1")
    is_training_mode = (mode == tf.estimator.ModeKeys.TRAIN)
    
     # 18 of the middle layers with Convolution, batch normalization and afterwards ReLu
    #conv2 = conv_layer(conv1, "conv2", is_training = is_training_mode)
    #conv3 = conv_layer(conv2, "conv3", is_training = is_training_mode)
    #conv4 = conv_layer(conv3, "conv4", is_training = is_training_mode)
    #conv5 = conv_layer(conv4, "conv5", is_training = is_training_mode)
    #conv6 = conv_layer(conv5, "conv6", is_training = is_training_mode)
    #conv7 = conv_layer(conv6, "conv7", is_training = is_training_mode)
    #conv8 = conv_layer(conv7, "conv8", is_training = is_training_mode)
    #conv9 = conv_layer(conv8, "conv9", is_training = is_training_mode)
    #conv10 = conv_layer(conv9, "conv10", is_training = is_training_mode)
    #conv11 = conv_layer(conv10, "conv11", is_training = is_training_mode)
    #conv12 = conv_layer(conv11, "conv12", is_training = is_training_mode)
    #conv13 = conv_layer(conv12, "conv13", is_training = is_training_mode)
    #conv14 = conv_layer(conv13, "conv14", is_training = is_training_mode)
    #conv15 = conv_layer(conv14, "conv15", is_training = is_training_mode)
    #conv16 = conv_layer(conv15, "conv16", is_training = is_training_mode)
    #conv17 = conv_layer(conv16, "conv17", is_training = is_training_mode)
    #conv18 = conv_layer(conv17, "conv18", is_training = is_training_mode)
    #conv19 = conv_layer(conv18, "conv19", is_training = is_training_mode)

    # final 
    final_layer = tf.layers.conv2d(
        inputs = conv1,
        filters = 1,
        kernel_size = [1, 1],
        padding = "same",
        activation = None,
        name = "final_layer") + input_layer
    
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions=final_layer)
    
    # Calculate Loss (for both Train and EVAL modes)
    # See that the residual learning is implemented here.
    loss = tf.losses.mean_squared_error(labels = labels , predictions = final_layer)
    tf.summary.scalar("Value_Loss_Function", loss)
        
    # Configure the Training OP (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        # calculate current learning rate:
        alpha = tf.train.get_global_step() / tau
        cur_learning_rate = (1-alpha) * eps_start + alpha * eps_end
        tf.summary.scalar("Learning rate", cur_learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate = cur_learning_rate)
        train_op = optimizer.minimize(loss = loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
        
        
    # Output all learnable variables for tensorboard
    for var in tf.trainable_variables():
        name = var.name
        name = name.replace(':', '_')
        tf.summary.histogram(name, var)
    merged_summary = tf.summary.merge_all()
    
    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.mean_squared_error(
            labels=labels, predictions=final_layer)}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)    


Load train and test data

In [ ]:
train_data = X_train
train_labels = Y_train
eval_data = X_eval
eval_labels = Y_eval

Create the estimator

In [ ]:
runconf = tf.estimator.RunConfig(save_summary_steps=1, log_step_count_steps = 2)

In [ ]:
ImpNet = tf.estimator.Estimator(config=runconf,
    model_fn=cnn_model_fn, model_dir="/tmp/tmp/test3"
    )

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

logging_hook = tf.train.SummarySaverHook(
     save_steps= 1,
    output_dir="/home/cloud/Training/summary_hook", 
    scaffold=tf.train.Scaffold(summary_op=tf.summary.merge_all())) 

Train the model

In [113]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=Y_train,
    batch_size=60,
    num_epochs=None,
    shuffle=True)

In [114]:
ImpNet.train(
    input_fn=train_input_fn,
    steps=20)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp/test2/model.ckpt-60
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 60 into /tmp/tmp/test2/model.ckpt.
INFO:tensorflow:loss = 0.010336229, step = 61
INFO:tensorflow:global_step/sec: 0.153005
INFO:tensorflow:loss = 0.012089028, step = 63 (13.077 sec)
INFO:tensorflow:global_step/sec: 0.172331
INFO:tensorflow:loss = 0.013791879, step = 65 (11.604 sec)
INFO:tensorflow:global_step/sec: 0.170142
INFO:tensorflow:loss = 0.01066425, step = 67 (11.758 sec)
INFO:tensorflow:global_step/sec: 0.177341
INFO:tensorflow:loss = 0.012131366, step = 69 (11.280 sec)
INFO:tensorflow:global_step/sec: 0.176181
INFO:tensorflow:loss = 0.012853254, step = 71 (11.350 sec)
INFO:tensorflow:global_step/sec: 0.174442
INFO:tensorflow:loss = 0.0

KeyboardInterrupt: 

## Evaluate the model

Make predictions

This is what we train for (our label)

In [ ]:
plt.imshow(Y_eval[1,:,:,0], cmap='gray' )

This is the input of the NN (The noisy image)

In [ ]:
plt.imshow(X_eval[1,:,:,0], cmap='gray' )

This is the true image, can be calculated as the difference of X to the label we train on

In [ ]:
plt.imshow(X_eval[1,:,:,0] - Y_eval[1,:,:,0] , cmap='gray' )

This is an original image. What does our network predict?

In [ ]:
# Evaluate the model and print results
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_eval[1:2,:,:,:]},
    y=X_eval[1:2,:,:,:],
    num_epochs=1,
    shuffle=False)
predict_results = ImpNet.predict(input_fn=predict_input_fn)
predict_results = list(predict_results)

The first dictionary is the image shown above. Although I cannot see much in the picture the network is 99.8% sure that it is original!

In [ ]:
plt.imshow(predict_results[0][:,:,0], cmap='gray')

In [ ]:
plt.imshow(X_eval[1:2,:,:,:].reshape([256, 256]) - predict_results[0][:,:,0], cmap='gray')